In [77]:
import os
import torch
import argparse
import numpy as np

from tqdm.auto import trange
from omegaconf import OmegaConf

from stable_baselines3.common.vec_env import DummyVecEnv

from trajectory.models.gpt import GPT
from trajectory.utils.common import set_seed
from trajectory.utils.env import create_env, rollout, vec_rollout

import gym

from citylearn.citylearn import CityLearnEnv
import itertools


In [78]:
from tqdm.auto import trange
from trajectory.planning.beam import beam_plan, batch_beam_plan


In [79]:
device = "cuda:1"

In [108]:
#checkpoints_path =  "checkpoints/city_learn/uniform/baseline"
checkpoints_path =  "checkpoints/city_learn/traj_config_v1/uniform/baseline"

In [109]:
#run_config = "checkpoints/city_learn/uniform/baseline/configs.yaml"
run_config = "configs/medium/city_learn_traj.yaml"
config = "configs/eval_base.yaml"
config = OmegaConf.load(config)

In [110]:
run_config = OmegaConf.load(run_config)

In [111]:
discretizer = torch.load(os.path.join(checkpoints_path, "discretizer.pt"), map_location=device)

In [112]:
model = GPT(**run_config.model)
model.eval()
model.to(device)

GPT(
  (tok_emb): Embedding(3300, 128)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0-3): 4 x TransformerBlock(
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.1, inplace=False)
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (head): EinLinear(n_models=33, in_features=128, out_features=100, bias=False)
)

In [113]:
total_params = sum(p.numel() for p in model.parameters())

In [114]:
total_params

1680384

In [115]:
model.load_state_dict(torch.load(os.path.join(checkpoints_path, "model_last.pt"), map_location=device))

<All keys matched successfully>

### Environment Settings

In [116]:
schema = "citylearn_challenge_2022_phase_1"

In [117]:
def action_space_to_dict(aspace):
    """ Only for box space """
    return { "high": aspace.high,
             "low": aspace.low,
             "shape": aspace.shape,
             "dtype": str(aspace.dtype)
    }

def env_reset(env):
    observations = env.reset()
    action_space = env.action_space
    observation_space = env.observation_space
    #building_info = env.get_building_information()
    #building_info = list(building_info.values())
    action_space_dicts = [action_space_to_dict(asp) for asp in action_space]
    observation_space_dicts = [action_space_to_dict(osp) for osp in observation_space]
    obs_dict = {"action_space": action_space_dicts,
                "observation_space": observation_space_dicts,
              #  "building_info": building_info,
                "observation": observations }
    return obs_dict

In [118]:
index_commun = [0, 2, 19, 4, 8, 24]
index_particular = [20, 21, 22, 23]

normalization_value_commun = [12, 24, 2, 100, 100, 1]
normalization_value_particular = [5, 5, 5, 5]

len_tot_index = len(index_commun) + len(index_particular) * 5

## env wrapper for stable baselines
class EnvCityGym(gym.Env):
    """
    Env wrapper coming from the gym library.
    """
    def __init__(self, env):
        self.env = env

        # get the number of buildings
        self.num_buildings = len(env.action_space)

        # define action and observation space
        self.action_space = gym.spaces.Box(low=np.array([-1] * self.num_buildings), high=np.array([1] * self.num_buildings), dtype=np.float32)

        # define the observation space
        self.observation_space = gym.spaces.Box(low=np.array([0] * len_tot_index), high=np.array([1] * len_tot_index), dtype=np.float32)

        # TO THINK : normalize the observation space
        self.current_obs = None
    def reset(self):
        obs_dict = env_reset(self.env)
        obs = self.env.reset()

        observation = self.get_observation(obs)
        
        self.current_obs = observation
        self.interactions = []

        return observation

    def get_observation(self, obs):
        """
        We retrieve new observation from the building observation to get a proper array of observation
        Basicly the observation array will be something like obs[0][index_commun] + obs[i][index_particular] for i in range(5)

        The first element of the new observation will be "commun observation" among all building like month / hour / carbon intensity / outdoor_dry_bulb_temperature_predicted_6h ...
        The next element of the new observation will be the concatenation of certain observation specific to buildings non_shiftable_load / solar_generation / ...  
        """
        
        # we get the observation commun for each building (index_commun)
        observation_commun = [obs[0][i]/n for i, n in zip(index_commun, normalization_value_commun)]
        observation_particular = [[o[i]/n for i, n in zip(index_particular, normalization_value_particular)] for o in obs]

        observation_particular = list(itertools.chain(*observation_particular))
        # we concatenate the observation
        observation = observation_commun + observation_particular

        return observation

    def step(self, action):
        """
        we apply the same action for all the buildings
        """
        # reprocessing action
        action = [[act] for act in action]

        # we do a step in the environment
        obs, reward, done, info = self.env.step(action)
        
        observation = self.get_observation(obs)
        
        
        self.interactions.append({
            "observations": self.current_obs,
            "next_observations": self.get_observation(obs),  # Assuming next observation is same as current for simplicity
            "actions": action,
            "rewards": reward,
            "dones": done,
            "info": info
        })
        
        self.current_obs = observation
        
        

        return observation, sum(reward), done, info
        
    def render(self, mode='human'):
        return self.env.render(mode)

In [119]:
env = CityLearnEnv(schema=schema)
env = EnvCityGym(env)

/home/ml-stud15/anaconda3/envs/stable2/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
/home/ml-stud15/anaconda3/envs/stable2/lib/python3.9/site-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [120]:
beam_context_size = 5 #config.beam_context 5
beam_width = 32 #config.beam_width 32
beam_steps = 3#config.beam_steps 5
plan_every = config.plan_every
sample_expand = config.sample_expand
k_act = config.k_act
k_obs = config.k_obs
k_reward = config.k_reward
temperature = config.temperature
discount = 0.99  #config.discount
max_steps = 300

In [121]:
temperature

1.0

In [122]:
value_placeholder = 1e6

In [123]:
transition_dim, obs_dim, act_dim = model.transition_dim, model.observation_dim, model.action_dim
    # trajectory of tokens for model action planning
    # +1 just to avoid index error while updating context on the last step
context = torch.zeros(1, model.transition_dim * (max_steps + 1), dtype=torch.long).to(device)

obs = env.reset()
obs = np.array(obs)
total_reward = 0
obs_tokens = discretizer.encode(obs, subslice=(0, obs_dim)).squeeze()

context[:, :model.observation_dim] = torch.as_tensor(obs_tokens, device=device)  # initial tokens for planning

In [124]:
for step in (pbar := trange(max_steps, desc="Rollout steps", leave=False)):
    #print(step)
    if step % plan_every == 0:
            # removing zeros from future, keep only context updated so far
        context_offset = model.transition_dim * (step + 1) - model.action_dim - 2
            # prediction: [a, s, a, s, ...]
        prediction_tokens = beam_plan(
                model, discretizer, context[:, :context_offset],
                steps=beam_steps,
                beam_width=beam_width,
                context_size=beam_context_size,
                k_act=k_act, k_obs=k_obs, k_reward=k_reward,
                temperature=temperature,
                discount=discount,
                sample_expand=sample_expand
            )
    else:
            # shift one transition forward in plan
        prediction_tokens = prediction_tokens[transition_dim:]
    action_tokens = prediction_tokens[:act_dim]
    action = discretizer.decode(action_tokens.cpu().numpy(), subslice=(obs_dim, obs_dim + act_dim)).squeeze()
    obs, reward, done, _ = env.step(action)
    obs = np.array(obs)
    
    total_reward +=reward
    
    pbar.set_postfix({'Reward': total_reward})
    if done:
        break
        
    obs_tokens = discretizer.encode(obs, subslice=(0, obs_dim)).squeeze()
    reward_tokens = discretizer.encode(
            np.array([reward, value_placeholder]),
            subslice=(transition_dim - 2, transition_dim)
        )
    # updating context with new action and obs
    context_offset = model.transition_dim * step
        # [s, ...] -> [s, a, ...]
    context[:, context_offset + obs_dim:context_offset + obs_dim + act_dim] = torch.as_tensor(action_tokens, device=device)
        # [s, a, ...] -> [s, a, r, v, ...]
    context[:, context_offset + transition_dim - 2:context_offset + transition_dim] = torch.as_tensor(reward_tokens, device=device)
        # [s, a, r, v, ...] -> [s, a, r, v, s, ...]
    context[:, context_offset + model.transition_dim:context_offset + model.transition_dim + model.observation_dim] = torch.as_tensor(obs_tokens, device=device)
    
    

Rollout steps:   0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [125]:
kpis = env.env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value').round(3)
kpis = kpis.dropna(how='all')
display(kpis)

name,Building_1,Building_2,Building_3,Building_4,Building_5,District
cost_function,,,,,,
annual_normalized_unserved_energy_total,0.000,0.000,0.000,0.000,0.000,0.000
annual_peak_average,NaN,NaN,NaN,NaN,NaN,1.131
carbon_emissions_total,1.113,1.061,1.138,1.535,1.081,1.186
cost_total,1.052,1.053,0.980,1.437,1.062,1.117
daily_one_minus_load_factor_average,NaN,NaN,NaN,NaN,NaN,0.958
daily_peak_average,NaN,NaN,NaN,NaN,NaN,1.024
discomfort_delta_average,0.000,0.000,0.000,0.000,0.000,0.000
discomfort_delta_maximum,0.000,0.000,0.000,0.000,0.000,0.000
discomfort_delta_minimum,0.000,0.000,0.000,0.000,0.000,0.000


In [76]:
df_evaluate[df_evaluate.cost_function=="electricity_consumption_total"]

,cost_function,value,name,level
12,electricity_consumption_total,1.162163,District,district
18,electricity_consumption_total,1.243585,Building_6,building
31,electricity_consumption_total,1.042793,Building_7,building
44,electricity_consumption_total,1.216176,Building_8,building
57,electricity_consumption_total,1.250840,Building_9,building
70,electricity_consumption_total,1.057421,Building_10,building
